In [2]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [2]:
with open('News.txt', encoding="utf8") as f:
    lines = f.read()

In [3]:
doc1 = nlp(lines)

for ent in doc1.ents:
    if ent.label_ == "GPE" :
        print(ent.text)

Ukraine
Ukraine
Kherson
Kherson
Russia
Ukraine
Russia
Ukraine
London
Russia
the west bank
Ukrainian
Kherson
Kakhova
Ukraine
Kherson
Zaporizhzhia
Ukraine
Moscow
US
Ukraine
agoGermany
Israel
Ukraine
Dagestan
UK


In [4]:
doc1.ents

(Ukraine,
 Russian,
 Dnipro River,
 Ukraine,
 The Guardian
 Skip,
 Guardian - Back,
 NewsWorld,
 crisisUkraineEnvironmentScienceGlobal,
 OpinionThe Guardian,
 CultureBooksMusicTV & radioArt & designFilmGamesClassicalStageLifestyleView,
 & sexHealth & fitnessHome & gardenWomenMenFamilyTravelMoneySearch,
 SearchSupport,
 ArchiveGuardian Puzzles,
 LicensingThe Guardian,
 appVideoPodcastsPicturesNewslettersToday,
 the GuardianThe ObserverGuardian,
 ArchiveGuardian Puzzles,
 Ukrainian,
 the Dnipro River,
 Kherson,
 last month,
 Alex Babenko/,
 the Dnipro River,
 Kherson,
 last month,
 Alex Babenko,
 Russian,
 Dnipro RiverBeachhead,
 Kherson,
 Russia,
 Tom Burgis,
 KhersonFri,
 GMTLast,
 Fri 3 Nov 2023 20.40,
 Ukrainian,
 the Dnipro River,
 Russian,
 Ukraine,
 Kherson,
 Ukrainian,
 Crimea,
 Russia,
 2014,
 two,
 Ukrainian,
 the east bank,
 mid-October,
 Russian,
 Ukrainian,
 Volodymyr Saldo,
 Putin,
 Kherson,
 Russian,
 Ukrainians,
 Kherson,
 Ukraine,
 Russian,
 Russian,
 fortificationsThis 

In [63]:
doc2 = nlp("the Israel-Hamas war")
doc3 = nlp("Anthony Albanese’s three-day tour of Shanghai and Beijing")

print(doc2.similarity(doc3))

0.5912129800317639


C:\Users\sures\AppData\Local\Temp\ipykernel_29416\1013727239.py:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print(doc2.similarity(doc3))


In [15]:
doc4 = """Computer: Based on the keywords in the title and body of the text, I would categorize the article as belonging to the "Politics" class.

The Guardian is a UK newspaper and they are using their own category scheme which is different from the one used by the user. The user has categorized the article under "Politics" but the Guardian has categorized it under "Israel". So there is a difference in how the article is being categorized by the two entities.. """

for tok in nlp(doc4):
    print(str(tok).lower())

computer
:
based
on
the
keywords
in
the
title
and
body
of
the
text
,
i
would
categorize
the
article
as
belonging
to
the
"
politics
"
class
.



the
guardian
is
a
uk
newspaper
and
they
are
using
their
own
category
scheme
which
is
different
from
the
one
used
by
the
user
.
the
user
has
categorized
the
article
under
"
politics
"
but
the
guardian
has
categorized
it
under
"
israel
"
.
so
there
is
a
difference
in
how
the
article
is
being
categorized
by
the
two
entities
..


# 2. Classify to broad dimentions

In [25]:
# Built a multilabel classification system using SpaCy (build and train the modek)

import pandas as pd

import sys
import os
import spacy
from spacy import displacy

from tqdm.auto import tqdm
from spacy.tokens import DocBin

In [3]:
def make_docs(train_data):

    train_data = train_data.drop(columns=["Id"])

    data = tuple(zip(train_data['Title'].tolist(), train_data['Label'].tolist())) 

    nlp = spacy.load("en_core_web_sm")

    docs = []

    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        if (label=='Nature'):
            doc.cats['Nature'] = 1
            doc.cats['Politics'] = 0
            doc.cats['Entertainment']  = 0
            doc.cats['Economics']  = 0
            doc.cats['Culture']  = 0
            doc.cats['Science']  = 0
        elif (label=='Politics'):
            doc.cats['Nature'] = 0
            doc.cats['Politics'] = 1
            doc.cats['Entertainment']  = 0
            doc.cats['Economics']  = 0
            doc.cats['Culture']  = 0
            doc.cats['Science']  = 0
        elif (label=='Entertainment'):
            doc.cats['Nature'] = 0
            doc.cats['Politics'] = 0
            doc.cats['Entertainment']  = 1
            doc.cats['Economics']  = 0
            doc.cats['Culture']  = 0
            doc.cats['Science']  = 0
        elif (label=='Economics'):
            doc.cats['Nature'] = 0
            doc.cats['Politics'] = 0
            doc.cats['Entertainment']  = 0
            doc.cats['Economics']  = 1
            doc.cats['Culture']  = 0
            doc.cats['Science']  = 0
        elif (label=='Culture'):
            doc.cats['Nature'] = 0
            doc.cats['Politics'] = 0
            doc.cats['Entertainment']  = 0
            doc.cats['Economics']  = 0
            doc.cats['Culture']  = 1
            doc.cats['Science']  = 0
        elif (label=='Science'):
            doc.cats['Nature'] = 0
            doc.cats['Politics'] = 0
            doc.cats['Entertainment']  = 0
            doc.cats['Economics']  = 0
            doc.cats['Culture']  = 0
            doc.cats['Science']  = 1        

        docs.append(doc)

    return docs,train_data

In [68]:
df = pd.read_csv("C:\Work\Programming\Hackathons\Diplomacy software\Dimensions.csv")
train = df.sample(frac=0.7, replace=False, random_state=1)

train_docs, train_data = make_docs(train)

doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("./textcat_data/textcat_train.spacy")

test = df.sample(frac=0.3, replace=False, random_state=1)

test_docs, train_data = make_docs(test)

doc_bin = DocBin(docs=test_docs)
doc_bin.to_disk("./textcat_data/textcat_valid.spacy")

100%|██████████| 12/12 [00:00<00:00, 512.69it/s]


In [69]:
train_data

,Title,Label
2,NBA Playoffs: Lakers and Clippers Face Off in ...,Entertainment
30,Exotic Destinations for Your Next Adventure Va...,Entertainment
3,President Signs Historic Climate Change Legisl...,Politics
21,Scientific Expedition Discovers New Species in...,Science
26,High School Robotics Team Wins International C...,Politics
28,Mental Health Awareness Month: Initiatives to ...,Science
22,Innovative Schools Implement Technology-Driven...,Science
36,Wildfires Threaten Communities in the Western ...,Nature
19,Researchers Develop Promising Treatment for Al...,Science
25,Education Policy Reforms Aim to Improve Studen...,Politics


In [70]:
!python -m spacy init fill-config ./textcat_base_config.cfg ./textcat_config.cfg

✔ Auto-filled config with all values
✔ Saved config
textcat_config.cfg
You can now add your data and train your pipeline:
python -m spacy train textcat_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [71]:
!python -m spacy train textcat_config.cfg --verbose --output ./textcat_output --paths.train textcat_data/textcat_train.spacy --paths.dev textcat_data/textcat_valid.spacy

[2023-10-27 22:02:30,992] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
[2023-10-27 22:02:31,180] [INFO] Set up nlp object from config
[2023-10-27 22:02:31,187] [DEBUG] Loading corpus from path: textcat_data\textcat_valid.spacy
[2023-10-27 22:02:31,192] [DEBUG] Loading corpus from path: textcat_data\textcat_train.spacy
[2023-10-27 22:02:31,192] [INFO] Pipeline: ['tok2vec', 'textcat']
[2023-10-27 22:02:31,192] [INFO] Created vocabulary
[2023-10-27 22:02:31,587] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_sm) has no vectors. This is almost certainly a mistake.
[2023-10-27 22:02:31,587] [INFO] Added vectors: en_core_web_sm
[2023-10-27 22:02:31,587] [INFO] Finished initializing nlp object
[2023-10-27 22:02:31,852] [INFO] Initialized pipeline components: ['tok2vec', 'textcat']
[2023-10-27 22:02:31,864] [DEBUG] Loading corpus from path: textcat_data\textcat_valid.spacy
[2023-10-27 22:02:31,864] [DEBUG] Loading corpus from path: textcat_data\

ℹ Saving to output directory: textcat_output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ------------  ----------  ------
  0       0          0.00          0.14       37.26    0.37
 77     200         84.36          1.09       66.67    0.67
177     400        482.64          0.23       66.67    0.67
277     600         19.61          0.01       66.67    0.67
418     800          0.00          0.00       66.67    0.67
618    1000          0.00          0.00       66.67    0.67
818    1200          0.00          0.00       66.67    0.67
1018    1400          0.00          0.00       66.67    0.67
1218    1600          0.00          0.00       66.67    0.67
1418    1800          0.00          0

In [28]:
prompt = input("Enter a headline to analyse: ")
nlp_textcat = spacy.load(os.getcwd() + "\\textcat_output\\model-last")
docPred = nlp_textcat(prompt)
res = docPred.cats
print(res)
print(prompt)
print("The class of this headline is", max(res, key=res.get))

{'Nature': 0.6959365010261536, 'Politics': 2.370913716731593e-05, 'Entertainment': 2.5340261000650344e-09, 'Economics': 0.1244477927684784, 'Culture': 0.00027904147282242775, 'Science': 0.17931297421455383}
Singapore votes in favour of UN resolution to protect civilians, uphold humanitarian obligations in Gaza Strip
The class of this headline is Nature


# 3. Map onto seriousness of risk

# 4. Differentiate domesting and international events

In [32]:
lines = 'Imagine, if you will, the daily routine of a tortoise in winter: every day you wake up under the dog’s bed and set off on a series of clockwise laps around a large kitchen table.\nYour visual field is wide but not very high – if the known universe has a ceiling, you have never seen it. The first three laps are flat and featureless, perfect repetitions of a cycle. But on the fourth lap half an apple suddenly appears, lying cut side up. How serendipitous! You think: I should probably eat that while I’m here, for who knows when I will next come across another … Wait, is that a grape rolling by in the distance?\nI’m sitting at the kitchen table, drinking coffee, reading headlines and listening to the rhythmic thunk of the tortoise circling, followed by a pause when he finds the apple I left in his path, then more thunking, and then another pause for the grape.\nAfter a few moments of silence I experience an unpleasant but familiar sensation: a spreading pool of tortoise piss reaching the edge of my sock.\n“Ugh,” I say, lifting my foot.\n“Fucking tortoise,” my wife says, entering the room, scooping him up and putting him outside the back door.\n“It’s too cold,” I say.\n“It’s not too cold,” she says. “It’s sunny.”\n“But it will be too cold when the sun goes down,” I say, “and then I’ll have to find him in the dark.”\n“That’s right,” my wife says. “You have one job.”\n“Bins,” I say, holding up two fingers.\n“Fine, you have two jobs.”\n“Then there’s my actual job,” I say.\n“And when were you thinking of making a start on that?” my wife says.\n“Right now,” I say, standing and closing my laptop.\n“Aren’t you going to clean this up first?” she says, pointing at the puddle of piss extending under the table.\nI say nothing, but I think: that’s four jobs.\nOn the way to my office shed I see the tortoise on wet grass in the shadow of the house, looking glum. But when I next look up, he has disappeared.\nI am hard at work when my wife opens my office door 20 minutes later – I saw her coming in time to put down my banjo and open my laptop.\n“Yes?” I say, typing nonsense. “Can I help you?”\n“Actually would you mind working in the house for a bit?” she says. “I’ve got to go out and there’s an important package about to be delivered.”\nI stop typing and spin my chair round to face her, arms folded.\n“How the tables have turned,” I say.\n“Please,” she says.\n“An important package, you say?”\n“It’s a new radiator,” she says. “For the bathroom.”\nWe once tried to solve this problem by installing a second wireless doorbell in my office, but it didn’t work out. It turns out many delivery people, when faced with two doorbells, will choose to knock.\n“Fine,” I say, following my wife back to the house.\nThe radiator arrives about 15 minutes later, but I decided not to go back to my office, because it’s warmer in the kitchen and it’s only an hour until lunch. Later that evening my wife enters the living room while I am watching the news.\n“I’m hungry,” she says.\n“I’ll start cooking in a minute,” I say, thinking: how many jobs is that? Five? Six?\nAfter supper a small dispute about what to watch on TV is settled by watching what my wife wants to watch – a film she abandons three-quarters of the way through.\n“I’m going to bed,” she says.\n“Now?” I say.\n“I’ve had a very busy day,” she says, watching me to see if I dare to say the same.\n“No comment,” I say.\nI stay to the end of the film, then turn off all the lights and head upstairs, where my wife is reading in bed. I remove one sock, then the other, while she watches me.\n“Did you do the bins?” she says. I put my socks back on.\nThe path to the kerb is piled with construction waste from the ripped-out bathroom, so I have to move the car out of the drive to get the bins to the street. I think: why does life have be so hard?\nBack upstairs I get undressed, brush my teeth, climb into bed and close my eyes. A minute later, when my wife turns out her reading lamp, my eyes snap open in the dark and I think: fucking tortoise.'

# with open('News.txt', encoding="utf8") as f:
#     lines = f.read()

In [33]:
doc1 = nlp(lines)

places = []

for ent in doc1.ents:
    if ent.label_ == "GPE" :
        places.append(ent.text)

print(places)

[]


In [34]:
# This blocks initialised the geography module
from functools import partial
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="diplomacy-software")

geocode = partial(geolocator.geocode, language="en")

In [35]:
countries = {}

for place in places:
    try:
        country = geocode(place)[0].split(',')[-1].strip()

        if country not in countries:
            countries[country] = 1
        else:
            countries[country] += 1

    except TypeError:
        pass

countries

{}

In [36]:
total = sum(list(countries.values()))

threshold = total / len(countries)

relevant = []

for country in countries:
    
    if countries[country] > threshold:
        relevant.append(country)

print(relevant)

ZeroDivisionError: division by zero

: 

In [21]:
countries

{'Ukraine': 14,
 'Russia': 5,
 'United Kingdom': 1,
 'United States': 1,
 'Czechia': 1,
 'Israel': 1,
 'Central African Republic': 1}